# OD-matrices creation (O-format type)

In [ ]:
import pandas as pd
import numpy as np

# Leggi il file CSV
data = pd.read_csv("matriceod_persone_viaggi_privati.csv", delimiter=';')
print(data)

# Converti la colonna NumViaggi da float a interi
data['NumViaggi'] = data['NumViaggi'].astype(float)

# Definizione delle percentuali per le 24 fasce orarie
percentuali = [0.8, 0.5, 0.4, 0.3, 0.4, 1.2, 4.5, 7.4, 6.6, 5.2, 5.0, 5.0, 5.2, 5.3, 5.6, 6.7, 8.4, 8.6, 7.4, 5.0, 3.9, 3.0, 2.1, 1.6]
percentuali_normalizzate = [x / sum(percentuali) for x in percentuali]

# Itera attraverso i dati
for i in range(24):
    mat_hour = data.copy()
    # Calcola il numero totale di viaggi per ogni coppia origine-destinazione
    #total_trips = data.groupby(['ORIG_COD_ZONA', 'DEST_COD_ZONA'])['NumViaggi'].sum()#.tolist()
    total_trips = mat_hour['NumViaggi'].tolist()
    
    # Moltiplica il numero totale di viaggi per ciascuna matrice OD per la percentuale corrispondente a ciascuna fascia oraria
    total_trips_per_hour = [int(x * percentuali_normalizzate[i]) for x in total_trips]
    
    # Intestazione per la matrice OD
    header = f"$OR;D2\n* From-Time To-Time\n{i}.00 {(i + 1) % 24}.00\n*Factor\n1.00\n* some\n* additional\n* comments\n"
    mat_hour['NumViaggi'] = total_trips_per_hour
    # Creazione della matrice OD
    with open(f"od_matrix_{i+1}.od", 'w') as file:
        # Scrivi l'intestazione nel file
        file.write(header)
        
        # Dati della matrice
        for j, (origine, destinazione) in enumerate(data.groupby(['ORIG_COD_ZONA', 'DEST_COD_ZONA']).groups.keys()):
            file.write(f"{origine:>4} {destinazione:>4} {total_trips_per_hour[j]:>4}\n")


In [ ]:
import pandas as pd
import numpy as np

# Leggi il file CSV
data = pd.read_csv("matriceod_persone_viaggi_pubblici.csv", delimiter=';')

# Converti la colonna NumViaggi da float a interi
data['NumViaggi'] = data['NumViaggi'].astype(float)

# Definizione delle percentuali per le 24 fasce orarie
percentuali = [0.3,0.4,0.4,0.6,0.8,2.0,4.8,7.5,9.0,8.7,9.0,9.0,7.5,8.4,7.8,6.9,5.4,4.0,2.7,1.8,1.2,0.9,0.6,0.3]
percentuali_normalizzate = [x / sum(percentuali) for x in percentuali]

# Itera attraverso i dati
for i in range(24):
    # Calcola il numero totale di viaggi per ogni coppia origine-destinazione
    total_trips = data.groupby(['ORIG_COD_ZONA', 'DEST_COD_ZONA'])['NumViaggi'].sum().tolist()
    
    # Moltiplica il numero totale di viaggi per ciascuna matrice OD per la percentuale corrispondente a ciascuna fascia oraria
    total_trips_per_hour = [int(x * percentuali_normalizzate[i]) for x in total_trips]
    
    # Intestazione per la matrice OD
    header = f"$OR;D2\n* From-Time To-Time\n{i}.00 {(i + 1) % 24}.00\n*Factor\n1.00\n* some\n* additional\n* comments\n"
    
    # Creazione della matrice OD
    with open(f"od_matrix_pt_{i+1}.od", 'w') as file:
        # Scrivi l'intestazione nel file
        file.write(header)
        
        # Dati della matrice
        for j, (origine, destinazione) in enumerate(data.groupby(['ORIG_COD_ZONA', 'DEST_COD_ZONA']).groups.keys()):
            file.write(f"{origine:>4} {destinazione:>4} {total_trips_per_hour[j]:>4}\n")